In [5]:
import os
import pandas as pd
import numpy as np

# キーワードリストを読み込む
keywords_df = pd.read_csv('output/Denso/Keywords1.csv', encoding='utf-8-sig')
keywords_list = keywords_df.iloc[:, 0].tolist()

# データを格納するリストを作成
data = []

# CSVファイルが入っているフォルダを指定
folder_path = 'output/Denso'

# フォルダ内のCSVファイルを全て取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('69020_デンソー_20241008-170737_Centrality_350.csv')]

# 各CSVファイルを処理
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    
    # ファイル名から企業コード、銘柄名を抽出
    parts = file.split('_')
    if len(parts) >= 4:
        company_code = parts[0]
        stock_name = parts[1]
    
        # 各Nodeについて順位付けを実施
        for centrality in ['Degree Centrality', 'Betweenness Centrality', 'Closeness Centrality', 'Eigenvector Centrality']:
            try:
                df[f'Rank_{centrality}'] = df[centrality].rank(ascending=False, method='min')
            except Exception as e:
                print(f"Error ranking centrality {centrality} in file {file}: {e}")
                df[f'Rank_{centrality}'] = np.nan
        
        # キーワードリストに含まれるNodeのレコードを抽出
        filtered_df = df[df['Node'].isin(keywords_list)]
        
        # 必要な情報をリストに追加
        for _, row in filtered_df.iterrows():
            data.append({
                '企業コード': company_code,
                '企業名': stock_name,
                'Node': row['Node'],
                'Degree Centrality': row['Degree Centrality'],
                'Betweenness Centrality': row['Betweenness Centrality'],
                'Closeness Centrality': row['Closeness Centrality'],
                'Eigenvector Centrality': row['Eigenvector Centrality'],
                'Rank_Degree Centrality': row['Rank_Degree Centrality'],
                'Rank_Betweenness Centrality': row['Rank_Betweenness Centrality'],
                'Rank_Closeness Centrality': row['Rank_Closeness Centrality'],
                'Rank_Eigenvector Centrality': row['Rank_Eigenvector Centrality'],
                'Community': row['Community']
            })

# リストからデータフレームを作成
output_df = pd.DataFrame(data, columns=[
    '企業コード', '企業名', 'Node', 'Degree Centrality', 'Betweenness Centrality', 
    'Closeness Centrality', 'Eigenvector Centrality',
    'Rank_Degree Centrality', 'Rank_Betweenness Centrality', 
    'Rank_Closeness Centrality', 'Rank_Eigenvector Centrality', 'Community'
])

# 幅の設定（例えば、幅を10に設定）
bin_width = 10

# 出力用のリストを作成
histogram_data = []

# 各企業コードと銘柄名ごとにデータを集計
for (company_code, stock_name), group in output_df.groupby(['企業コード', '企業名']):
    # 各Centralityについてヒストグラムを作成
    for centrality in ['Rank_Degree Centrality','Rank_Betweenness Centrality', 'Rank_Closeness Centrality', 'Rank_Eigenvector Centrality']:
        # 順位が計算されていない場合はスキップ
        if group[centrality].isnull().all():
            continue
        
        try:
            # 順位の範囲を作成
            max_rank = group[centrality].max()
            bins = np.arange(0, max_rank + bin_width, bin_width)
            
            # ヒストグラムの計算
            counts, bin_edges = np.histogram(group[centrality].dropna(), bins=bins)
            
            # ヒストグラムの結果をリストに追加
            for i in range(len(counts)):
                histogram_data.append({
                    'コード': company_code,
                    '企業名': stock_name,
                    '順位範囲': f'{bin_edges[i]}-{bin_edges[i+1]-1}',
                    f'順位個数_{centrality.split("_")[-1]}': counts[i]
                })
        except Exception as e:
            print(f"Error creating histogram for centrality {centrality} in group {company_code}_{stock_name}: {e}")

# リストからヒストグラムのデータフレームを作成
histogram_df = pd.DataFrame(histogram_data)

# Define columns that would be in the pivoted DataFrame
pivot_columns = [
    'コード', '企業名', '順位範囲', 
    '順位個数_Degree Centrality', 
    '順位個数_Betweenness Centrality', 
    '順位個数_Closeness Centrality', 
    '順位個数_Eigenvector Centrality'
]

# Check if histogram_df is None or empty
if histogram_df is None or histogram_df.empty:
    # Create an empty DataFrame with the correct columns
    pivoted_histogram_df = pd.DataFrame(columns=pivot_columns)
else:
    # Perform the pivot operation
    pivoted_histogram_df = histogram_df.pivot_table(
        index=['コード', '企業名', '順位範囲'], 
        values=[
            '順位個数_Degree Centrality', 
            '順位個数_Betweenness Centrality', 
            '順位個数_Closeness Centrality', 
            '順位個数_Eigenvector Centrality'
        ], 
        aggfunc='sum', 
        fill_value=0
    ).reset_index()

# ヒストグラムの結果をCSVファイルに保存
pivoted_histogram_df.to_csv('output/Denso/centrality_histogram_IP.csv', index=False, encoding='utf-8-sig')
